Neuroevolution: Exercise 6
=========
###### Artur Ganzha 10019651
---------	
###### Raul Gorek 10061333
---------	

In [1]:
import numpy as np
### NeuralNet
def derivative_bcel(prediction, ground_truth):
    x =  np.where(ground_truth == 0, 1.0 / (1.0 - prediction), -1.0 / prediction)
    return x

def derivative_mse(prediction, ground_truth):
    batch_size = ground_truth.shape[0]
    return -2 * (ground_truth - prediction) / batch_size

class Linear:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.W = np.random.uniform(-1, 1,(self.input_size,self.output_size))
        self.B = np.zeros((1, self.output_size))
    
    def forward(self, x):
        self.fw = x
        return np.dot(x, self.W) + self.B
    
    def backward(self, d, lr):
        d_w = np.dot(self.fw.T, d)
        d_e = np.dot(d, self.W.T)
        d_b = np.sum(d, axis=0, keepdims=True)
        self.W -= lr * d_w / self.fw.shape[0]
        self.B -= lr * d_b / self.fw.shape[0]
        return d_e


class ReLU:
    def __init__(self):
        pass

    def forward(self, x):
        self.fw = x
        return x * (x > 0)
    
    def backward(self, d, lr):
        return d * np.where(self.fw > 0, 1.0, 0.0)
    

class Sigmoid:
    def __init__(self):
        pass
    
    def forward(self, x):
        self.fw = x
        self.out = 1.0 / (1.0 + np.exp(-x))
        return self.out
    
    def backward(self, d, lr):
        return d * (self.out * (1.0 - self.out))
    

class NeuralNetwork:
    def __init__(self, layers: list):
        self.layers = layers

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward_pass(self, deriv, lr):
        for layer in reversed(self.layers):
            deriv = layer.backward(deriv, lr)

# Aufgabe 1

In [2]:
import copy
class Indiviuum:
    def __init__(self, n_obs, n_act):
        self.net = NeuralNetwork([
            Linear(n_obs, 32), ReLU(),
            Linear(32, 32), ReLU(),
            Linear(32, n_act)
        ])
        self.fitness = -np.inf

    def mutate(self, rate):
        if np.random.random() < rate:
            for layer in self.net.layers:
                if type(layer) == Linear:
                    layer.W += np.random.normal(0,1,size=layer.W.shape)
                    layer.B += np.random.normal(0,1,size=layer.B.shape)

    def eval(self, func: callable):
        self.fitness = func(self)

    def copy(self):
        return copy.deepcopy(self)

# Aufgabe 2

In [3]:
import gymnasium as gym
import time

env = gym.make('CartPole-v1')
num_actions = env.action_space.n
obs_shape = env.observation_space.shape[0]

In [4]:
NUM_EVAL_FIT = 5
MAX_TIME_BUDGET = 5
def fitness(ind: Indiviuum):
    rewards = np.zeros((NUM_EVAL_FIT,))
    for i in range(NUM_EVAL_FIT):
        end = False
        obs, _ = env.reset()
        t_start = time.perf_counter()
        while not end or time.perf_counter() - t_start > MAX_TIME_BUDGET:
            action = np.argmax(ind.net.forward_pass(obs))
            obs, r, ter, trunc, _ = env.step(action)
            rewards[i] += r
            end = ter or trunc
    return np.mean(rewards)

In [5]:
MAX_TIME_BUDGET = 5
def run100(ind: Indiviuum):
    return np.mean([fitness(ind) for _ in range(int(100.0 / NUM_EVAL_FIT))])

Warum ist es besser ein Individuum mehrmals zu testen: Da eine gewisse Zufallskomponente in dem Environment liegt, wird nicht jeder Durchlauf gleich sein. D.h. der summierte Reward folgt einer Verteilung. Daher ist es besser den Durchschnitt über ein paar wenige Durchläufe zu nehmen (bei zu vielen Durchläufen könnte es strärker auf die Rechenzeit gehen.)

In [6]:
ind = Indiviuum(obs_shape, num_actions)
ind.eval(fitness)
print(ind.fitness)
ind2 = ind.copy()
print(ind2.fitness)

9.0
9.0


In [7]:
def render(ind: Indiviuum):
    tenv = gym.make('CartPole-v1', render_mode='human')
    print(ind.fitness)
    obs, _ = tenv.reset()
    end = False
    while not end:
        tenv.render()
        action = np.argmax(ind.net.forward_pass(obs))
        obs, _, ter, trunc, _ = tenv.step(action)
        end = ter or trunc

    env.close()

#### Mit Elitist

In [8]:
POP_SIZE = 100
MAX_GEN = 200
SELECTION_SIZE = 20
MUT_RATE_DECAY = 0.9

def EA_elite():
    # Init
    population = [Indiviuum(obs_shape, num_actions) for _ in range(POP_SIZE)]
    mr = 1.0
    # Check terminal
    generation = 0
    max_f = 0
    while generation < MAX_GEN and max_f < 475:
        # Selection
        for ind in population: ind.eval(fitness)
        population = sorted(population, key=lambda x: x.fitness, reverse=True)
        selected = population[:SELECTION_SIZE]
        max_f = run100(selected[0])
        print(f"Generation: {generation+1} Max fitness over 100 runs: {max_f}")
        # Mutation
        mutated = []
        i = 0
        while len(mutated) < POP_SIZE - SELECTION_SIZE:
            copied = selected[i].copy()
            copied.mutate(mr)
            mutated.append(copied)
        
        population = selected + mutated
        generation += 1
    mr *= MUT_RATE_DECAY

    for ind in population: ind.eval(fitness)
    population = sorted(population, key=lambda x: x.fitness, reverse=True)
    return population[0], generation

In [9]:
POP_SIZE = 100
MAX_GEN = 200
SELECTION_SIZE = 20
MUT_RATE_DECAY = 0.9

def EA_prop():
    # Init
    population = [Indiviuum(obs_shape, num_actions) for _ in range(POP_SIZE)]
    mr = 1.0
    # Check terminal
    generation = 0
    max_f = 0
    while generation < MAX_GEN and max_f < 475:
        # Selection
        for ind in population: ind.eval(fitness)
        population = sorted(population, key=lambda x: x.fitness, reverse=True)
        f = np.array([i.fitness for i in population])
        selected = list(np.random.choice(population, size=(SELECTION_SIZE,), p=f/f.sum(), replace=False))
        selected = sorted(selected, key=lambda x: x.fitness, reverse=True)
        max_f = run100(selected[0])
        print(f"Generation: {generation+1} Max fitness over 100 runs: {max_f}")
        # Mutation
        mutated = []
        i = 0
        while len(mutated) < POP_SIZE - SELECTION_SIZE:
            copied = selected[i].copy()
            copied.mutate(mr)
            mutated.append(copied)
        
        population = selected + mutated
        generation += 1
    mr *= MUT_RATE_DECAY

    for ind in population: ind.eval(fitness)
    population = sorted(population, key=lambda x: x.fitness, reverse=True)
    return population[0], generation

In [10]:
best, _ = EA_elite()
f100 = run100(best)
print(f'Mean of 100 runs: {f100}')
render(best)

Generation: 1 Max fitness over 100 runs: 226.41
Generation: 2 Max fitness over 100 runs: 498.55
Mean of 100 runs: 500.0
500.0


In [11]:
best, _ = EA_prop()
f100 = run100(best)
print(f'Mean of 100 runs: {f100}')
render(best)

Generation: 1 Max fitness over 100 runs: 366.68999999999994
Generation: 2 Max fitness over 100 runs: 383.13
Generation: 3 Max fitness over 100 runs: 370.25
Generation: 4 Max fitness over 100 runs: 162.59999999999997
Generation: 5 Max fitness over 100 runs: 253.61999999999998
Generation: 6 Max fitness over 100 runs: 274.43999999999994
Generation: 7 Max fitness over 100 runs: 267.25000000000006
Generation: 8 Max fitness over 100 runs: 278.63999999999993
Generation: 9 Max fitness over 100 runs: 263.06
Generation: 10 Max fitness over 100 runs: 280.85
Generation: 11 Max fitness over 100 runs: 473.66
Generation: 12 Max fitness over 100 runs: 472.51000000000005
Generation: 13 Max fitness over 100 runs: 478.36
Mean of 100 runs: 498.57
500.0


In [12]:
from IPython.display import clear_output
NUM = 100
genmeanprop = np.zeros((NUM,))
genmeanelite = np.zeros((NUM,))
for i in range(NUM):
    _, genp = EA_prop()
    _, gene = EA_elite()
    genmeanprop[i] = genp
    genmeanelite[i] = gene
    clear_output(wait=True)
    print(i)

print(f'Durchschnittliche Generationen Elitist: {np.mean(genmeanelite)}')
print(f'Durchschnittliche Generationen Proportional: {np.mean(genmeanprop)}')

99
Durchschnittliche Generationen Elitist: 10.98
Durchschnittliche Generationen Proportional: 11.04


In den Experimenten zeigt sich, dass elitist selection ein mini bisschen besser funktioniert, als Fitness proportional selection. Aber beides kovergiert im Durchschnitt nach 11 Generationen